<a href="https://colab.research.google.com/github/Tikuh5/2024_Geoinformatique_I/blob/main/surface_agricole_utile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas pandas pooch

import pooch



In [2]:
# URL du fichier ZIP
url = "https://files.swisscarto.ch/geoinf1/vd_dgav_surfaces_agricoles.zip"

# Définir le chemin d'enregistrement du fichier ZIP
filename = pooch.retrieve(url, known_hash=None)

# Décompresser l'archive
import zipfile
import os

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('/content/surfaces_agricoles')

import geopandas as gpd



SHA256 hash of downloaded file: 0d866067c647f235d92499e669b8dfb4dfea18da3cfd99434a7e6176a9fba81f
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


In [3]:
# Charger le Geopackage contenant la couche "agr_dgav_sau"
gpk_path = "/content/surfaces_agricoles/vd_dgav_surfaces_agricoles.gpkg"
gdf_agriculture = gpd.read_file(gpk_path, layer='agr_dgav_sau')

from google.colab import files

In [4]:
# Télécharger le fichier depuis votre ordinateur
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# Lire le fichier Geopackage téléchargé
filename = list(uploaded.keys())[0]  # Récupérer le nom du fichier téléchargé
gdf_perimetre = gpd.read_file(filename)  # Charger le fichier dans GeoPandas

# Intersection des polygones des surfaces agricoles avec le périmètre
gdf_agriculture_intersected = gpd.overlay(gdf_agriculture, gdf_perimetre, how='intersection')

import pandas as pd

# Lire le fichier CSV
csv_path = '/content/surfaces_agricoles/vd_dgav_sau_utilisation_2023.csv'
df_utilisation = pd.read_csv(csv_path)

# Effectuer la jointure sur la colonne 'ident' pour ajouter toutes les informations du CSV
sau_data = gdf_agriculture_intersected.merge(df_utilisation, left_on='ident', right_on= 'IDENT', how='left')

# Calcul de la superficie en hectares (l'unité est en mètres carrés par défaut)
sau_data['surface_ha'] = sau_data.geometry.area / 10000

# Regrouper par utilisation et calculer la somme des surfaces
resume_surfaces = sau_data.groupby('UTILISATIO')['surface_ha'].sum().reset_index()

# Sauvegarder le résumé dans un fichier CSV
resume_surfaces.to_csv('/content/resume_surfaces_agricoles.csv', index=False)

# Sauvegarder le GeoDataFrame dans un Geopackage
del(sau_data['IDENT'])
sau_data.to_file('/content/surfaces_agricoles_utilisation.gpkg', layer='surfaces_agricoles_utilisation', driver="GPKG")

from google.colab import files

# Télécharger les fichiers générés
files.download('/content/resume_surfaces_agricoles.csv')
files.download('/content/surfaces_agricoles_utilisation.gpkg')